In [ ]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
import sys
from pylab import *
import os
import urllib.request
import itertools
import seaborn as sns
import numpy as np
from adjustText import adjust_text
import statistics

In [ ]:
#get league info - requires league to be PUBLIC viewable
league_id = #insertleagueid here
year = #insertyear
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + str(year)+'/segments/0/leagues/'+str(league_id)
#cookies = {"swid":"{53C447FA-F80D-447C-A228-31DA5528E1A5}",
#          "espn_s2":"AEALJB1U8dzv47STesAjeGBieNUNmSuCg77rSQAIktcgegMhWUHoQixvwz58wTJsdpyYqdyAIeJpYjIikklqyNH0MToI7LbrN3EZWez4DbXFbnrKm39ys%2Bn3%2BKXmA7IPRSOMAQFipO3SeCcw6WDX%2B8%2Bw9NMV9xcKLGyXdJYQPSYrbAzwNn4qqEfx8pkh8SYs49ISW8n%2FWZW7JHu10hMf14QPYkEafTiNZeCXwx4y4f4Z3PcJU%2FFirwhtB3AG0Apu3WzMXJuPTmxWmLgHM6XaJCa%2BOXHcoBc6%2FdMLzmU1Txrkyg%3D%3D"}
r = requests.get(url)#, cookies)

#get json from espn fantasy football API
d = r.json()

#get team info
teams = d['teams']
#example from first team
#teams[0]

In [ ]:
#get matchups from each week
r = requests.get(url, params={"view":"mMatchupScore"})
schedule = r.json()
#schedule

In [ ]:
#create basic dataframe for scores from each matchup each week
weeklyScores = [[
        game['matchupPeriodId'],
        game['home']['teamId'], game['home']['totalPoints'],
        game['away']['teamId'], game['away']['totalPoints']
        ] for game in schedule['schedule']]
weeklyScores = pd.DataFrame(weeklyScores, columns=['Week', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
weeklyScores['Type'] = ['Regular' if w<= 13 else 'Playoff' for w in weeklyScores['Week']]
#weeklyScores.head()

In [ ]:
#for post-final week of regular season only
#weeklyScores = []
#72 may change based on number of matchups per week*regular season weeks
#for game in range(0, 72):
#    if game < 72:
#        weeklyScores.append([
#                schedule['schedule'][game]['matchupPeriodId'],
#                schedule['schedule'][game]['home']['teamId'], schedule['schedule'][game]['home']['totalPoints'],
#                schedule['schedule'][game]['away']['teamId'], schedule['schedule'][game]['away']['totalPoints']
#                ]) #for game in schedule['schedule']]
#weeklyScores = pd.DataFrame(weeklyScores, columns=['Week', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
#weeklyScores['Type'] = ['Regular' if w<= 13 else 'Playoff' for w in weeklyScores['Week']]
#weeklyScores.head()

In [ ]:
#attach team names to their matchup and score

teamName1 = []
teamName2 = []

#for every row, go through teams json view and append names to list
for index, row in weeklyScores.iterrows():
    for team in teams:
        if row['Team 1'] == team['id']:
            teamName1.append(team['location'] + ' ' + team['nickname'])
            
for index, row in weeklyScores.iterrows():
    for team in teams:
        if row['Team 2'] == team['id']:
            teamName2.append(team['location'] + ' ' + team['nickname'])

#put names from list in appropriate column
weeklyScores['Team Name 1'] = teamName1
weeklyScores['Team Name 2'] = teamName2

#re-order columns using column_order list
column_order = ["Week", "Team 1", "Team Name 1", "Score 1", "Team 2", "Team Name 2", "Score 2"]
weeklyScores = weeklyScores.reindex(columns=column_order)
#weeklyScores.head(12)

In [ ]:
#get team overall info from json for standings
r = requests.get(url, params={"view":"mTeam"})
teamstandings = r.json()

#example of overall info for a team
teamstandings['teams'][0]['record']['overall']
#teamstandings

In [ ]:
#create standings dataframe
standings = []
    
standings = [[
        team['id'],
        team['location'] + ' ' + team['nickname'], 
        team['record']['overall']['wins'], team['record']['overall']['losses'], team['record']['overall']['ties'], team['record']['overall']['percentage'], 
        team['record']['overall']['pointsFor'], team['record']['overall']['pointsAgainst']
        ] for team in teamstandings['teams']]
standings = pd.DataFrame(standings, columns=['Team ID', 'Team Name', 'Wins', 'Losses', 'Ties', 'Win %', 'Points For', 'Points Against'])
#standings

In [ ]:
#add analytical columns to standings dataframe

#pythagorean wins
standings['Expected Wins'] = ((standings['Points For']**2)/((standings['Points For']**2)+(standings['Points Against']**2)))*(standings['Wins'] + standings['Losses'] + standings['Ties'])
standings['Expected Win %'] = standings['Expected Wins']/(standings['Wins'] + standings['Losses'] + standings['Ties'])

standings['Points Per Week'] = standings['Points For']/(standings['Wins'] + standings['Losses'] + standings['Ties'])

#start median wins at points per week, will be calced later
standings['Deserved Wins'] = 0
standings['Deserved Win %'] = standings['Deserved Wins']/(standings['Wins'] + standings['Losses'] + standings['Ties'])

#round columns appropriately
standings['Expected Wins'] = standings['Expected Wins'].round(2)
standings['Points Per Week'] = standings['Points Per Week'].round(2)
standings['Win %'] = standings['Win %'].round(3)
standings['Expected Win %'] = standings['Expected Win %'].round(3)

#rearrange columns
column_order = ["Team ID", "Team Name", "Wins", "Losses", "Ties", "Win %", "Points For", "Points Against", "Points Per Week", "Expected Wins", "Expected Win %"]
standings = standings.reindex(columns=column_order)
#standings

In [ ]:
#included here to prevent accidental increases from re-running cell
standings['Deserved Wins'] = 0

totalWeeks = standings.loc[(0)]['Wins'] + standings.loc[(0)]['Losses'] + standings.loc[(0)]['Ties']
totalWeeks

weeklyMedian = []

#get weekly median in list for each week
for i in range(1, totalWeeks+1):
    currentWeekScores = pd.concat([weeklyScores["Score 1"][weeklyScores['Week'] == i], weeklyScores["Score 2"][weeklyScores['Week'] == i]]).sort_index().values.tolist()
    #print(currentWeekScores)
    weeklyMedian.append(statistics.median(currentWeekScores))
#    print('Week ' + str(i) + ': ' + str(weeklyMedian))

#add a deserved win for each team when they have a score over the weekly median
for index, row in weeklyScores.iterrows():
    for id in standings['Team ID']:
        if row['Team 1'] == id:
            #ensure we stay within length of weeklyMedian
            if (row['Week'] <= len(weeklyMedian)):
                #use -1 for correct index
                if (row['Score 1'] > weeklyMedian[row['Week']-1]):
                    standings.loc[(standings['Team ID'] == id) & (standings['Team ID'] == row['Team 1']), 'Deserved Wins'] += 1

        elif row['Team 2'] == id: 
            #ensure we stay within length of weeklyMedian
            if (row['Week'] <= len(weeklyMedian)):
                #use -1 for correct index
                if (row['Score 2'] > weeklyMedian[row['Week']-1]):
                    standings.loc[(standings['Team ID'] == id) & (standings['Team ID'] == row['Team 2']), 'Deserved Wins'] += 1
        else:
            continue

#standings

In [ ]:
#set appropriate deserved win %
standings['Deserved Win %'] = standings['Deserved Wins']/(standings['Wins'] + standings['Losses'] + standings['Ties'])
standings['Deserved Win %'] = standings['Deserved Win %'].round(3)
#standings

In [ ]:
#set power ranking values
standings['Power Rank Score'] = ((standings['Points For'])*40 + (standings['Points For'] * standings['Expected Win %'])*20 + (standings['Points For'] * standings['Deserved Win %'])*40)/100
standings['Power Rank Score'] = standings['Power Rank Score'].round(1)
#standings.sort_values(by='Power Rank Score', ascending=False)

In [ ]:
finalStandings = standings[['Team Name', 'Wins', 'Losses', 'Ties', 'Win %', 'Points For', 'Points Per Week', 'Expected Wins', 'Deserved Wins', 'Deserved Win %', 'Power Rank Score']]
finalStandings = finalStandings.sort_values(by='Power Rank Score', ascending=False)
#finalStandings

In [ ]:
import dataframe_image as dfi
finalStandings.dfi.export('FFLStandings.png')

In [ ]:
#standings['Max Score'] = 0
#standings['Min Score'] = 9999
#standings

In [ ]:
#finding max and min scores
#for id in standings['Team ID']:
#    maxScore = 0
#    minScore = 9999
#    for index, row in weeklyScores.iterrows():
#        if row['Team 1'] == id:
#            #ensure we stay within length of weeklyMedian
#            if row['Score 1'] > maxScore:
#                maxScore = row['Score 1']
#            if (row['Score 1'] < minScore) & (row['Score 1'] > 0):
#                minScore = row['Score 1']
#        elif row['Team 2'] == id:
#            #ensure we stay within length of weeklyMedian
#            if row['Score 2'] > maxScore:
#                maxScore = row['Score 2']
#            if (row['Score 2'] < minScore) & (row['Score 2'] > 0):
#                minScore = row['Score 2']
#        else:
#            continue
#    standings.loc[(standings['Team ID'] == id), 'Max Score'] = maxScore  
#    standings.loc[(standings['Team ID'] == id), 'Min Score'] = minScore
#standings